# Filtering your DataFrame

- In the previous exercise, you have subset the data using `select()` operator which is mainly used to subset the DataFrame column-wise. What if you want to subset the DataFrame based on a condition (for example, select all rows where the sex is Female). In this exercise, you will filter the rows in the `people_df` DataFrame in which `'sex'` is `female` and `male` and create two different datasets. Finally, you'll count the number of rows in each of those datasets.

- Remember, you already have `SparkSession` `spark` and `people_df` DataFrame available in your workspace.


## Instructions

- Filter the `people_df` DataFrame to select all rows where `sex` is `female` into `people_df_female` DataFrame.
- Filter the `people_df` DataFrame to select all rows where `sex` is `male` into `people_df_male` DataFrame.
- Count the number of rows in `people_df_female` and `people_df_male` DataFrames.

In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
file_path = "file:////home/talentum/test-jupyter/P2/M1/SM9/Dataset/people.csv"

# Create an DataFrame from file_path
people_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Filter people_df to select females 
people_df_female = people_df.select(people_df.sex == "female")

# Filter people_df to select males
people_df_male = people_df.filter(people_df.sex == "male")

# Count the number of rows 
print("There are {} rows in the people_df_female DataFrame and {} rows in the people_df_male DataFrame".format(people_df_female.count(), people_df_male.count()))

There are 100000 rows in the people_df_female DataFrame and 49066 rows in the people_df_male DataFrame


In [8]:
file_path = "file:///home/talentum/test-jupyter/P2/M1/SM9/Dataset/people.csv"

people_df = spark.read.csv(file_path, header=True, inferSchema=True)

people_df_sub = people_df.select('name', 'sex', 'date of birth')

people_df_sub_count = people_df_sub.groupBy('name', 'sex', 'date of birth').count()

people_df_sub_duplicates = people_df_sub_count.filter("count > 1")  

people_df_sub_duplicates.show()

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [9]:
people_df.groupBy(people_df.name, people_df.sex, people_df.date of birth) # error due to date of birth spaces we 
                                                                    #we cant use dot notation here

SyntaxError: invalid syntax (<ipython-input-9-86ff7e78e04a>, line 1)

In [13]:
# making it column object  people_df['date of birth']

people_df.groupBy(people_df.name, people_df.sex, people_df['date of birth'])\
.count()\
.filter(people_df.count>1).show() 



TypeError: condition should be string or Column

In [14]:
# making it column object  people_df['date of birth']

people_df.groupBy(people_df.name, people_df.sex, people_df['date of birth'])\
.count()\
.filter('count>1').show() 



+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [16]:
import pyspark.sql.functions as F

In [21]:
people_df.groupBy(people_df.name, people_df.sex, people_df['date of birth'])\
.count()\
.filter(F.col('count') > 1).show() 

print(type(F.col('count')))   #returns  column object

print(type(F.col('count') > 1))

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>


In [25]:
file_path = "file:///home/talentum/test-jupyter/P2/M1/SM9/Dataset/people.csv"

people_df = spark.read.csv(file_path, header=True, inferSchema=True)

people_df_sub = people_df.select('name', 'sex', 'date of birth')

people_df_sub_nodup = people_df_sub.dropDuplicates()

people_df_sub_duplicates = people_df_sub.exceptAll(people_df_sub_nodup)

people_df_sub_duplicates.show()

+-------------+------+-------------+
|         name|   sex|date of birth|
+-------------+------+-------------+
|Kathryn Davis|female|  20175-02-28|
| Robert Smith|  male|  20175-02-28|
+-------------+------+-------------+

